# 📊 Notebook 3 — RAG Pipeline Evaluation

This notebook evaluates the **Smart Document Assistant** RAG pipeline across four dimensions:

| Dimension | Method |
|-----------|--------|
| ✅ System Readiness | Checks that documents are ingested and chain is ready |
| 🔍 Retrieval Quality | Tests that relevant queries return non-empty answers |
| 📐 Answer Scoring | Fuzzy word-overlap score + exact contains check |
| 🛡️ Guardrails | Validates that input/output guardrails fire correctly |

> **How to use this notebook:**
> 1. First **ingest your documents** via the Gradio UI or `frontend.py`
> 2. Update the `test_cases` list in Cell 3 with questions **from your actual documents**
> 3. Run all cells top to bottom

---
## Cell 1 — Setup & Imports

In [1]:
import sys
import os
import time
import json

# ── Fix working directory ───────────────────────────────
# core.py uses relative paths like 'data/vectorstore'.
# When running from notebooks/, those paths break.
# We must cd to the project root so everything resolves.
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
os.chdir(PROJECT_ROOT)
sys.path.insert(0, PROJECT_ROOT)
print(f"Working directory: {os.getcwd()}")

from core import (
    get_rag_chain,
    get_ingested_doc_names,
    IngestionPipeline,
    Guardrails,
    Config
)

print("✅ Imports successful")
print(f"   LLM Provider  : {Config.LLM_PROVIDER}")
print(f"   Embedding Model: {Config.EMBEDDING_MODEL_NAME}")
print(f"   Vectorstore   : {os.path.abspath(Config.VECTORSTORE_PATH)}")

Working directory: e:\ITI\Level 2 (Advanced)\Final_Project


INFO:core:Using HuggingFace embeddings: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:core:Reloading ChainBuilder...
INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.
INFO:core:Loaded FAISS vectorstore from data/vectorstore, total vectors: 11
INFO:core:Retriever created with k=20 (fetch_k for post-filter). Total vectors: 11


✅ Imports successful
   LLM Provider  : groq
   Embedding Model: all-MiniLM-L6-v2
   Vectorstore   : e:\ITI\Level 2 (Advanced)\Final_Project\data\vectorstore


---
## Cell 2 — System Readiness Check

In [2]:
# Check what documents are currently ingested
doc_names = get_ingested_doc_names()

pipeline = IngestionPipeline()
vectorstore = pipeline.load_vectorstore()

print("=" * 55)
print("  SYSTEM READINESS CHECK")
print("=" * 55)

if not doc_names:
    print("❌ No documents found in the vectorstore!")
    print("   Please ingest documents first via frontend.py")
    print("   Then re-run this notebook.")
else:
    total_vectors = vectorstore.index.ntotal if vectorstore and hasattr(vectorstore, 'index') else 'N/A'
    print(f"✅ Documents indexed : {len(doc_names)}")
    print(f"✅ Total vectors     : {total_vectors}")
    print(f"✅ Chunk size        : {Config.CHUNK_SIZE}")
    print(f"✅ Top-K retrieval   : {Config.TOP_K}")
    print()
    print("  Ingested documents:")
    for i, name in enumerate(doc_names, 1):
        print(f"    {i}. {name}")

print("=" * 55)

INFO:core:Reloading ChainBuilder...
INFO:core:Loaded FAISS vectorstore from data/vectorstore, total vectors: 11
INFO:core:Retriever created with k=20 (fetch_k for post-filter). Total vectors: 11
INFO:core:Using HuggingFace embeddings: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:core:Loaded FAISS vectorstore from data/vectorstore, total vectors: 11


  SYSTEM READINESS CHECK
✅ Documents indexed : 1
✅ Total vectors     : 11
✅ Chunk size        : 1000
✅ Top-K retrieval   : 4

  Ingested documents:
    1. Hatsuyuki Sakura reveiew .docx


---
## Cell 3 — Define Your Test Cases

> **⚠️ IMPORTANT**: Edit the `test_cases` list below with questions **relevant to your actual uploaded documents**.
>
> - `question` → what you ask the RAG system
> - `ground_truth` → a key phrase or sentence you expect to appear in the answer  
> - `doc_filter` → *(optional)* restrict search to one specific document filename
>
> Leave `ground_truth` as `""` if you just want to see the answer without scoring.

In [3]:
# ============================================================
# ✏️  EDIT THESE TEST CASES TO MATCH YOUR DOCUMENTS
# ============================================================
test_cases = [
    {
        "question"    : "What is this document about?",
        "ground_truth": "",          # Empty = no scoring, just show the answer
        "doc_filter"  : None         # None = search all documents
    },
    {
        "question"    : "Summarize the main points of the document.",
        "ground_truth": "",
        "doc_filter"  : None
    },
    {
        "question"    : "What are the key terms or definitions mentioned?",
        "ground_truth": "",
        "doc_filter"  : None
    },
    # ── Add your own questions below ──────────────────────
    # {
    #     "question"    : "Your question here?",
    #     "ground_truth": "A phrase you expect in the answer",
    #     "doc_filter"  : "specific_file.pdf"   # or None
    # },
]

print(f"✅ {len(test_cases)} test cases defined.")
print()
for i, tc in enumerate(test_cases, 1):
    gt = tc['ground_truth'] if tc['ground_truth'] else "(no scoring — open-ended)"
    filt = tc['doc_filter'] if tc['doc_filter'] else "all documents"
    print(f"  [{i}] Q: {tc['question']}")
    print(f"       Expected: {gt}")
    print(f"       Scope   : {filt}")
    print()

✅ 3 test cases defined.

  [1] Q: What is this document about?
       Expected: (no scoring — open-ended)
       Scope   : all documents

  [2] Q: Summarize the main points of the document.
       Expected: (no scoring — open-ended)
       Scope   : all documents

  [3] Q: What are the key terms or definitions mentioned?
       Expected: (no scoring — open-ended)
       Scope   : all documents



---
## Cell 4 — Helper: Scoring Functions

In [4]:
def contains_score(ground_truth: str, answer: str) -> float:
    """
    Exact contains check (case-insensitive).
    Returns 1.0 if ground_truth appears anywhere in answer, else 0.0.
    """
    if not ground_truth:
        return None  # No ground truth = not scored
    return 1.0 if ground_truth.strip().lower() in answer.strip().lower() else 0.0


def word_overlap_score(ground_truth: str, answer: str) -> float:
    """
    Fuzzy word-overlap score (similar to ROUGE-1 recall).
    Measures what fraction of ground-truth words appear in the answer.
    Range: 0.0 – 1.0
    """
    if not ground_truth:
        return None
    gt_words  = set(ground_truth.lower().split())
    ans_words = set(answer.lower().split())
    if not gt_words:
        return 0.0
    overlap = gt_words & ans_words
    return round(len(overlap) / len(gt_words), 3)


def answer_length_score(answer: str, min_words: int = 10) -> float:
    """
    Checks whether the answer is substantive (not empty or too short).
    Returns 1.0 if answer has >= min_words words, else 0.0.
    """
    words = answer.strip().split()
    return 1.0 if len(words) >= min_words else 0.0


print("✅ Scoring functions ready")
print("   - contains_score()      → 1.0 if ground truth phrase found in answer")
print("   - word_overlap_score()  → fuzzy overlap (0.0–1.0)")
print("   - answer_length_score() → 1.0 if answer >= 10 words")

✅ Scoring functions ready
   - contains_score()      → 1.0 if ground truth phrase found in answer
   - word_overlap_score()  → fuzzy overlap (0.0–1.0)
   - answer_length_score() → 1.0 if answer >= 10 words


---
## Cell 5 — Run Evaluation Loop

In [5]:
eval_results = []  # Stores all per-question results

print("=" * 60)
print("  RUNNING EVALUATION")
print("=" * 60)

for i, case in enumerate(test_cases, 1):
    question     = case["question"]
    ground_truth = case.get("ground_truth", "")
    doc_filter   = case.get("doc_filter", None)

    print(f"\n[{i}/{len(test_cases)}] Question: {question}")
    if doc_filter:
        print(f"       Doc filter: {doc_filter}")
    print("-" * 55)

    try:
        # Build chain (with optional doc scope filter)
        chain = get_rag_chain(doc_filter=doc_filter)

        # Time the call
        t_start  = time.time()
        response = chain.invoke({"question": question})
        elapsed  = round(time.time() - t_start, 2)

        # Parse response
        if isinstance(response, dict):
            answer     = response.get("answer", "")
            citations  = response.get("citations", [])
            guardrails = response.get("guardrails", {})
            in_guard   = guardrails.get("input", {})
            out_guard  = guardrails.get("output", {})
        else:
            answer     = str(response)
            citations  = []
            in_guard   = {}
            out_guard  = {}

        # Compute scores
        c_score  = contains_score(ground_truth, answer)
        wo_score = word_overlap_score(ground_truth, answer)
        al_score = answer_length_score(answer)

        # Print answer
        print(f"  📝 Answer ({len(answer.split())} words, {elapsed}s):")
        # Print answer wrapped at ~80 chars per line
        words = answer.split()
        line, lines = [], []
        for w in words:
            line.append(w)
            if len(" ".join(line)) > 78:
                lines.append("     " + " ".join(line))
                line = []
        if line:
            lines.append("     " + " ".join(line))
        print("\n".join(lines[:8]))  # Show up to 8 lines
        if len(lines) > 8:
            print(f"     ... [{len(words) - 80} more words]")

        # Print citations
        if citations:
            print(f"\n  📎 Citations ({len(citations)}):")
            for c in citations:
                src  = c.source if hasattr(c, 'source') else c.get('source', '?')
                page = c.page   if hasattr(c, 'page')   else c.get('page', '?')
                print(f"       - {src} (page {page})")

        # Print guardrails
        print(f"\n  🛡️  Guardrails:")
        print(f"       Input  passed : {in_guard.get('passed', 'N/A')}  | score: {in_guard.get('score', 'N/A')}")
        grounded = out_guard.get('grounded', 'N/A')
        out_score = out_guard.get('score', 'N/A')
        print(f"       Output grounded: {grounded}  | score: {out_score}")
        if out_guard.get('warning'):
            print(f"       ⚠️  {out_guard['warning']}")

        # Print scores
        print(f"\n  📐 Scores:")
        print(f"       Answer length OK      : {'✅' if al_score == 1.0 else '❌'} ({len(answer.split())} words)")
        if c_score is not None:
            print(f"       Contains ground truth : {'✅' if c_score == 1.0 else '❌'} ({c_score})")
            print(f"       Word-overlap score    : {wo_score:.3f}  (ROUGE-1 recall)")
            if ground_truth:
                print(f"       Expected phrase       : '{ground_truth}'")
        else:
            print(f"       (no ground truth provided — open-ended question)")

        # Store result
        eval_results.append({
            "question"       : question,
            "answer"         : answer,
            "ground_truth"   : ground_truth,
            "doc_filter"     : doc_filter,
            "elapsed_s"      : elapsed,
            "num_citations"  : len(citations),
            "input_passed"   : in_guard.get('passed', None),
            "input_score"    : in_guard.get('score', None),
            "output_grounded": grounded,
            "output_score"   : out_score,
            "contains_score" : c_score,
            "word_overlap"   : wo_score,
            "length_score"   : al_score,
            "error"          : None
        })

    except Exception as e:
        print(f"  ❌ ERROR: {e}")
        eval_results.append({
            "question": question, "answer": "",
            "error": str(e),
            "contains_score": 0.0, "word_overlap": 0.0, "length_score": 0.0
        })

print()
print("=" * 60)
print(f"  Evaluation complete — {len(eval_results)} questions processed")
print("=" * 60)

INFO:core:Reloading ChainBuilder...
INFO:core:Loaded FAISS vectorstore from data/vectorstore, total vectors: 11
INFO:core:Retriever created with k=20 (fetch_k for post-filter). Total vectors: 11
INFO:core:Guardrail input check: similarity=0.357, threshold=0.25, passed=True
INFO:core:Retrieved 11 raw documents for query: 'What is this document about?'
INFO:core:  Retrieved doc_names: ['Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx']
INFO:core:  After deduplication: 11 unique docs
INFO:core:  Final top_k=4 docs selected
INFO:core:    Doc 0: doc_name=Hatsuyuki Sakura reveiew .docx, page=0, chunk_id=Hatsuyuki Sakura reveiew .docx_chunk_2
INFO:core:    Doc 1: doc_name=Hat

  RUNNING EVALUATION

[1/3] Question: What is this document about?
-------------------------------------------------------


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:core:Guardrail output check: similarity=0.496, threshold=0.2, grounded=True
INFO:core:Reloading ChainBuilder...
INFO:core:Loaded FAISS vectorstore from data/vectorstore, total vectors: 11
INFO:core:Retriever created with k=20 (fetch_k for post-filter). Total vectors: 11
INFO:core:Guardrail input check: similarity=0.375, threshold=0.25, passed=True
INFO:core:Retrieved 11 raw documents for query: 'Summarize the main points of the document.'
INFO:core:  Retrieved doc_names: ['Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx']
INFO:core:  After deduplication: 11 unique docs


  📝 Answer (34 words, 0.84s):
     This document appears to be a review of a visual novel or game called "Hatsuyuki
     Sakura". The review discusses the game's story, characters, and routes, providing
     an analysis of the game's strengths and weaknesses.

  📎 Citations (4):
       - Hatsuyuki Sakura reveiew .docx (page 0)
       - Hatsuyuki Sakura reveiew .docx (page 0)
       - Hatsuyuki Sakura reveiew .docx (page 0)
       - Hatsuyuki Sakura reveiew .docx (page 0)

  🛡️  Guardrails:
       Input  passed : True  | score: 0.3566383719444275
       Output grounded: True  | score: 0.49552791940586377

  📐 Scores:
       Answer length OK      : ✅ (34 words)
       (no ground truth provided — open-ended question)

[2/3] Question: Summarize the main points of the document.
-------------------------------------------------------


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:core:Guardrail output check: similarity=0.589, threshold=0.2, grounded=True
INFO:core:Reloading ChainBuilder...
INFO:core:Loaded FAISS vectorstore from data/vectorstore, total vectors: 11
INFO:core:Retriever created with k=20 (fetch_k for post-filter). Total vectors: 11
INFO:core:Guardrail input check: similarity=0.359, threshold=0.25, passed=True
INFO:core:Retrieved 11 raw documents for query: 'What are the key terms or definitions mentioned?'
INFO:core:  Retrieved doc_names: ['Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx', 'Hatsuyuki Sakura reveiew .docx']
INFO:core:  After deduplication: 11 unique

  📝 Answer (198 words, 1.09s):
     The document is a review of the visual novel "Hatsuyuki Sakura". Here are the main
     points summarized: 1. The game revolves around Kawano Hatsuyuki, who is revealed
     to be the Ghost Child, and his desire to avenge his beloved Ran's death. 2. The
     common route is well-written and sets up the groundwork for the story. 3. The game
     has 5 girls and 6 routes, with each route exploring a different character's relationship
     with Hatsuyuki. 4. The reviewer enjoyed Aya's route the most, finding it to be the
     best route in the game, with good plot relevancy and an interesting character. 5.
     The reviewer was disappointed with Shirokuma's route, finding it to be the shortest
     ... [118 more words]

  📎 Citations (4):
       - Hatsuyuki Sakura reveiew .docx (page 0)
       - Hatsuyuki Sakura reveiew .docx (page 0)
       - Hatsuyuki Sakura reveiew .docx (page 0)
       - Hatsuyuki Sakura reveiew .docx (page 0)

  🛡️  Guardrails:
   

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:core:Guardrail output check: similarity=0.299, threshold=0.2, grounded=True


  📝 Answer (144 words, 0.86s):
     Based on the provided context, the following key terms or definitions are mentioned:
     1. **Ghost Child**: Refers to Kawano Hatsuyuki, the main character, who is revealed
     to have this identity by the end of the common route. 2. **反魂香 (Hankoukou)**: A
     term mentioned in the context, but its meaning is not explicitly defined. However,
     based on the context, it seems to be related to a concept or a plot element in the
     game. 3. **Soul Fragmentation**: A term mentioned in the context, which seems to
     be related to a plot element or a mechanic in the game. 4. **Progress Guidance Committee
     ( 進路指導委員)**: A group mentioned in the context, which seems to be a part of the game's
     ... [64 more words]

  📎 Citations (4):
       - Hatsuyuki Sakura reveiew .docx (page 0)
       - Hatsuyuki Sakura reveiew .docx (page 0)
       - Hatsuyuki Sakura reveiew .docx (page 0)
       - Hatsuyuki Sakura reveiew .docx (page 0)

  🛡️  Guardrails

---
## Cell 6 — Summary Report

In [6]:
# ── Aggregate Metrics ────────────────────────────────────
total   = len(eval_results)
errors  = sum(1 for r in eval_results if r.get("error"))
success = total - errors

# Length scores (all questions)
length_scores = [r["length_score"] for r in eval_results if r.get("length_score") is not None]
avg_length_score = round(sum(length_scores) / len(length_scores), 3) if length_scores else 0.0

# Contains / word-overlap (only scored questions)
scored = [r for r in eval_results if r.get("contains_score") is not None and r.get("ground_truth")]
if scored:
    avg_contains  = round(sum(r["contains_score"] for r in scored) / len(scored), 3)
    avg_overlap   = round(sum(r["word_overlap"]   for r in scored) / len(scored), 3)
else:
    avg_contains  = None
    avg_overlap   = None

# Latency
latencies = [r["elapsed_s"] for r in eval_results if r.get("elapsed_s") and not r.get("error")]
avg_latency = round(sum(latencies) / len(latencies), 2) if latencies else 0.0
max_latency = max(latencies) if latencies else 0.0

# Input guardrail pass rate
in_results = [r["input_passed"] for r in eval_results if r.get("input_passed") is not None]
guard_pass_rate = round(sum(1 for v in in_results if v) / len(in_results), 3) if in_results else None

# Output grounding rate
out_results = [r["output_grounded"] for r in eval_results if r.get("output_grounded") is not None and r["output_grounded"] != 'N/A']
grounding_rate = round(sum(1 for v in out_results if v) / len(out_results), 3) if out_results else None

# ── Print Report ─────────────────────────────────────────
print()
print("╔" + "═" * 53 + "╗")
print("║       📊  EVALUATION SUMMARY REPORT                ║")
print("╠" + "═" * 53 + "╣")
print(f"║  Total questions          : {total:<25}║")
print(f"║  Successful invocations   : {success:<25}║")
print(f"║  Errors                   : {errors:<25}║")
print("╠" + "═" * 53 + "╣")
print("║  RETRIEVAL & ANSWER QUALITY                        ║")
print("╠" + "═" * 53 + "╣")
print(f"║  Avg answer length score  : {avg_length_score:<25}║")
if avg_contains is not None:
    print(f"║  Contains-GT score (avg)  : {avg_contains:<25}║")
    print(f"║  Word-overlap score (avg) : {avg_overlap:<25}║")
    print(f"║  (based on {len(scored)} scored question(s))             ║")
else:
    print(f"║  Contains-GT score        : N/A (no ground truths)   ║")
print("╠" + "═" * 53 + "╣")
print("║  GUARDRAILS                                        ║")
print("╠" + "═" * 53 + "╣")
gr_str = f"{grounding_rate:.1%}" if grounding_rate is not None else "N/A"
gp_str = f"{guard_pass_rate:.1%}" if guard_pass_rate is not None else "N/A"
print(f"║  Input guardrail pass rate: {gp_str:<25}║")
print(f"║  Output grounding rate    : {gr_str:<25}║")
print("╠" + "═" * 53 + "╣")
print("║  LATENCY                                           ║")
print("╠" + "═" * 53 + "╣")
print(f"║  Avg response time        : {avg_latency}s{'':<23}║")
print(f"║  Max response time        : {max_latency}s{'':<23}║")
print("╚" + "═" * 53 + "╝")

# Per-question table
print()
print("  Per-Question Results:")
print(f"  {'#':<3} {'Question (truncated)':<35} {'Length':>7} {'Overlap':>8} {'Time':>6}")
print("  " + "-" * 63)
for i, r in enumerate(eval_results, 1):
    q_short   = r['question'][:33] + ".." if len(r['question']) > 33 else r['question']
    ls        = f"{r['length_score']:.1f}" if r.get('length_score') is not None else "err"
    wo        = f"{r['word_overlap']:.3f}" if r.get('word_overlap') is not None else " N/A"
    t         = f"{r['elapsed_s']}s"        if r.get('elapsed_s')   else "err"
    print(f"  {i:<3} {q_short:<35} {ls:>7} {wo:>8} {t:>6}")


╔═════════════════════════════════════════════════════╗
║       📊  EVALUATION SUMMARY REPORT                ║
╠═════════════════════════════════════════════════════╣
║  Total questions          : 3                        ║
║  Successful invocations   : 3                        ║
║  Errors                   : 0                        ║
╠═════════════════════════════════════════════════════╣
║  RETRIEVAL & ANSWER QUALITY                        ║
╠═════════════════════════════════════════════════════╣
║  Avg answer length score  : 1.0                      ║
║  Contains-GT score        : N/A (no ground truths)   ║
╠═════════════════════════════════════════════════════╣
║  GUARDRAILS                                        ║
╠═════════════════════════════════════════════════════╣
║  Input guardrail pass rate: 100.0%                   ║
║  Output grounding rate    : 100.0%                   ║
╠═════════════════════════════════════════════════════╣
║  LATENCY                                  

---
## Cell 7 — Guardrail Stress Test

Directly tests the guardrail mechanisms with known on-topic and off-topic queries.

In [7]:
pipeline    = IngestionPipeline()
vectorstore = pipeline.load_vectorstore()
embeddings  = pipeline.embeddings

guardrail_tests = [
    # (label, query, expect_to_pass)
    ("On-topic (general)",  "What is this document about?",                                True),
    ("On-topic (specific)", "What are the main points mentioned?",                         True),
    ("Off-topic (food)",    "What is the recipe for chocolate cake with vanilla frosting?", False),
    ("Off-topic (weather)", "What is the weather forecast for Cairo tomorrow?",             False),
    ("Gibberish",           "asdfghjkl qwerty zxcvbnm",                                    False),
]

print("=" * 60)
print("  🛡️  GUARDRAIL STRESS TEST")
print("=" * 60)

if not vectorstore:
    print("❌ No vectorstore — skipping guardrail tests.")
else:
    guard_results = []
    for label, query, expect_pass in guardrail_tests:
        result  = Guardrails.input_check(query, vectorstore, embeddings)
        passed  = result.get("passed", False)
        score   = result.get("score",  0.0)
        correct = (passed == expect_pass)
        status  = "✅" if correct else "❌"
        print(f"  {status}  [{label}]")
        print(f"       Query   : {query[:60]}")
        print(f"       Score   : {score:.4f}  |  Passed: {passed}  |  Expected: {expect_pass}")
        if result.get("message"):
            print(f"       Message : {result['message'][:80]}")
        print()
        guard_results.append(correct)

    correct_count = sum(guard_results)
    print(f"  Guardrail accuracy: {correct_count}/{len(guard_results)} ({100*correct_count//len(guard_results)}%)")

print("=" * 60)

INFO:core:Using HuggingFace embeddings: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:core:Loaded FAISS vectorstore from data/vectorstore, total vectors: 11
INFO:core:Guardrail input check: similarity=0.357, threshold=0.25, passed=True
INFO:core:Guardrail input check: similarity=0.398, threshold=0.25, passed=True
INFO:core:Guardrail input check: similarity=0.366, threshold=0.25, passed=True


  🛡️  GUARDRAIL STRESS TEST
  ✅  [On-topic (general)]
       Query   : What is this document about?
       Score   : 0.3566  |  Passed: True  |  Expected: True

  ✅  [On-topic (specific)]
       Query   : What are the main points mentioned?
       Score   : 0.3975  |  Passed: True  |  Expected: True

  ❌  [Off-topic (food)]
       Query   : What is the recipe for chocolate cake with vanilla frosting?
       Score   : 0.3662  |  Passed: True  |  Expected: False



INFO:core:Guardrail input check: similarity=0.351, threshold=0.25, passed=True
INFO:core:Guardrail input check: similarity=0.357, threshold=0.25, passed=True


  ❌  [Off-topic (weather)]
       Query   : What is the weather forecast for Cairo tomorrow?
       Score   : 0.3509  |  Passed: True  |  Expected: False

  ❌  [Gibberish]
       Query   : asdfghjkl qwerty zxcvbnm
       Score   : 0.3573  |  Passed: True  |  Expected: False

  Guardrail accuracy: 2/5 (40%)


---
## Cell 8 — Save Results to JSON

In [8]:
import datetime

output_dir = os.path.join("data", "eval_results")
os.makedirs(output_dir, exist_ok=True)

timestamp   = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
output_path = os.path.join(output_dir, f"eval_{timestamp}.json")

report = {
    "timestamp"          : timestamp,
    "llm_provider"       : Config.LLM_PROVIDER,
    "embedding_model"    : Config.EMBEDDING_MODEL_NAME,
    "documents_indexed"  : doc_names,
    "total_questions"    : total,
    "avg_length_score"   : avg_length_score,
    "avg_contains_score" : avg_contains,
    "avg_word_overlap"   : avg_overlap,
    "avg_latency_s"      : avg_latency,
    "max_latency_s"      : max_latency,
    "guardrail_pass_rate": guard_pass_rate,
    "output_grounding_rate": grounding_rate,
    "questions"          : eval_results
}

# Make result serializable
def make_serializable(obj):
    if isinstance(obj, dict):
        return {k: make_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [make_serializable(v) for v in obj]
    elif hasattr(obj, '__dict__'):
        return make_serializable(obj.__dict__)
    else:
        try:
            json.dumps(obj)
            return obj
        except:
            return str(obj)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(make_serializable(report), f, indent=2, ensure_ascii=False)

print(f"✅ Results saved to: {os.path.abspath(output_path)}")
print(f"   File size: {os.path.getsize(output_path)} bytes")

✅ Results saved to: e:\ITI\Level 2 (Advanced)\Final_Project\data\eval_results\eval_20260219_020211.json
   File size: 4029 bytes


---
## 📋 Metric Reference

| Metric | How It's Computed | Good Score |
|--------|------------------|------------|
| **Answer Length Score** | `1.0` if answer ≥ 10 words, else `0.0` | `1.0` |
| **Contains-GT Score** | `1.0` if ground-truth phrase found inside answer | `1.0` |
| **Word-Overlap Score** | Fraction of ground-truth words that appear in answer (ROUGE-1 recall) | `> 0.5` |
| **Input Guardrail Score** | `1 / (1 + L2_distance)` vs the nearest FAISS chunk | `> 0.25` |
| **Output Grounding Score** | Cosine similarity between `embed(answer)` and `embed(context)` | `> 0.20` |
| **Response Time** | Wall-clock seconds from `chain.invoke()` to result | `< 10s` |

---

## ⚠️ Limitations of This Evaluation

1. **String-match scoring is brittle** — a perfect rephrased answer scores 0.0 if no exact words overlap.
2. **No semantic scoring** — for true semantic evaluation, consider integrating [RAGAS](https://github.com/explodinggradients/ragas) or an LLM judge.
3. **Ground truth must be hand-curated** — there is no automatic Q/A generation in this notebook.
4. **Single-turn only** — multi-turn conversation quality is not evaluated here.